In [1]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

El votante representativo se preocupa por el valor esperado de su función de utilidad  $\mathbb{E}_{t}^{P}(\Gamma_{t})$

<ul>
    <li>c representa el consumo del bien privado por parte de un ciudadano representativo.</li>
    <li>g es el consumo del bien público (per cápita)</li>
    <li>k es la inversión pública (per cápita)</li>
</ul>

Función de utilidad del líder $\mathbf{E}_{t}^{I}(\Gamma_{t})+\sum_{s=t}^{T}\beta^{s-t}X\pi_{s,t}$

Función de elección del líder

\begin{equation}
    v_{t} =
    \begin{cases}
        1 &  \text{si } \mathbf{E}_{t}^{P}(\Gamma_{t+1}) \geq \mathbf{E}_{t}^{P}(\Gamma_{t+1}^{O})\\
        0 & \text{en otro caso}\\
\end{cases}
\end{equation}

## Equilibrio bajo información completa

\begin{equation}
    \begin{aligned}
        \max_{\tau, g} & U(y-\tau, g)+\beta V(\tau + \epsilon - g) \\
        \text{sujeto a } & g, y- \tau, \tau + \epsilon-g \geq 0 \\
   \end{aligned}
\end{equation}

### En $t=\bar{t}$

In [8]:
# Parámetros
a, b = 1, 1
beta = 0.95

y = 100  # Valor exógeno

$$ \epsilon_{t}^{i} = \alpha_{t}^{i} + \alpha_{t-1}^{i}$$
$$ \eta_{t}^{i} = q_{t}^{i} + q_{t-1}^{i}$$

$$U(c, g) = a \ln{c} + b\ln{g}$$
$$V(k) = -\ln{k}$$

### Problema de maximizacion en T periodos de tiempo

In [51]:
# Parametros
T = 10
y = 100
beta = 0.95

a, b = 1, 1

In [52]:
# Probabilidad de que el agente i sea competente en el periodo t 
p = 0.5
# Rango de valores donde viven los shocks de apariencia
q_r = range(-10,10) 

In [53]:
# Valores aleatorios
alpha1_i = np.random.binomial(1, p, T-t)
alpha2_i = np.random.binomial(1, p, T-t)

alpha1_o = np.random.binomial(1, p, T-t)
alpha2_o = np.random.binomial(1, p, T-t)

epsilon_i = alpha1_i + alpha2_i # competencia del Incumbente independiente entre agentes y en el tiempo
epsilon_o = alpha1_o + alpha2_o # competencia del Oponente independiente entre agentes y en el tiempo

# -------------------------------------------------------------------------------------------------------------------
q1_i = np.random.choice(q_r, T-t)
q2_i = np.random.choice(q_r, T-t)

q1_o = np.random.choice(q_r, T-t)
q2_o = np.random.choice(q_r, T-t)

eta_i = q1_i + q2_i # Shock no relacionado con la competencia pero si con el liderazgo del Incumbente independiente entre agentes y en el tiempo
eta_o = q1_o + q2_o # Shock no relacionado con la competencia pero si con el liderazgo del Oponente independiente entre agentes y en el tiempo

In [48]:
# Definimos la funcion objetivo
def funcion_objetivo(x, y, epsilon, beta, a, b):
    tau, g = x
    c = y - tau
    k = tau + epsilon - g
    return -(a * np.log(c) + b * np.log(g) + beta * (-np.log(k)))  # Negativo porque usamos un minimizador de scipy

# Restricciones (Constraints)
def constraint1(x, y):
    # y - tau >= 0
    return y - x[0]

def constraint2(x):
    # g >= 0
    return x[1]

def constraint3(x, epsilon):
    # tau + epsilon - g >= 0
    return x[0] + epsilon - x[1]

In [54]:
# Guess inicial (Por ahora tau y g empiezan tomando valores en la mitad de su rango permitido)
x0 = [y/2, y/4]

______________________

## Parte dinamica

In [55]:
# Funcion para optimizar en cada periodo de tiempo
def optimizador_cada_t(T, y, beta, a, b, epsilon):
    tau_optimo = []
    g_optimo = []
    valores_max = []

    for t in range(T):
        # Itera sobre los valores de competencia del agente
        epsilon_t = epsilon[t]

        # Restricciones por cada periodo de tiempo(epsilon)
        con1 = {'type': 'ineq', 'fun': constraint1, 'args': (y,)}
        con2 = {'type': 'ineq', 'fun': constraint2}
        con3 = {'type': 'ineq', 'fun': constraint3, 'args': (epsilon_t,)}

        # Combinamos las restricciones en una lista
        constraints_t = [con1, con2, con3]

        # Optimiza para ese periodo de tiempo t
        resultado_t = minimize(funcion_objetivo, x0, args=(y, epsilon_t, beta, a, b), constraints=constraints_t)

        # Guarda los resultados para cada periodo de tiempo
        tau_optimo.append(resultado_t.x[0])
        g_optimo.append(resultado_t.x[1])
        valores_max.append(-resultado_t.fun)

    return tau_optimo, g_optimo, valores_max

In [65]:
# Optimiza T veces para 
tau_i_optimos_lista, g_i_optimos_lista, utilidades_i_maximos = optimizador_cada_t(T, y, beta, a, b, epsilon_i)

# Valor esperado
tau_i_esperado = np.mean(tau_i_optimos_lista)
g_i_esperado = np.mean(g_i_optimos_lista)

utilida_esperada_i = np.mean(utilidades_i_maximos)

C:\Users\oscar\AppData\Local\Temp\ipykernel_16688\3790639713.py:6: RuntimeWarning: invalid value encountered in log
  return -(a * np.log(c) + b * np.log(g) + beta * (-np.log(k)))  # Negativo porque usamos un minimizador de scipy


In [66]:
utilida_esperada_i

30.19217726303544

In [67]:
g_esperado_i

NameError: name 'g_esperado_i' is not defined

In [60]:
tau_esperado

38.88330556844336